In [ ]:
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
import yfinance as yf
pd.options.mode.chained_assignment = None

In [ ]:
#Defininfo o período de referência para treinamento do modelo
acao = "VALE3.SA"
inicio = "2015-12-31"
fim = "2024-04-10"
dados_acao = yf.download(acao, inicio, fim)

In [ ]:
#Pegando os dados da cotação no ato do fechamento, não na cotação ajustada
cotacao = dados_acao['Close'].to_numpy().reshape(-1, 1)

In [ ]:
#Definindo para serem treinados apenas 80% dos dados evitando o sobreajuste (overfitting)
dados_sendo_treinados = int(len(cotacao) * 0.8)
type(dados_sendo_treinados)

In [ ]:
#Escalonando os dados para uma escala entre 0 e 1 facilitando o treinamento da rede
escalador_dados = MinMaxScaler(feature_range=(0, 1))
dados_0_1_treinamento = escalador_dados.fit_transform(cotacao[0: dados_sendo_treinados, :])
dados_0_1_testes = escalador_dados.transform(cotacao[dados_sendo_treinados: , :])
dados_0_1 = list(dados_0_1_treinamento.reshape(len(dados_0_1_treinamento))) + list(dados_0_1_testes.reshape(len(dados_0_1_testes)))
dados_0_1 = np.array(dados_0_1).reshape(len(dados_0_1), 1)


In [ ]:
dados_a_treinar = dados_0_1[0: dados_sendo_treinados, :]

#Dados que irão gerar o resultado (cotações dos 60 dias passados)
treinamento_x = []
#Cotação real 
treinamento_y = []


for i in range(60, len(dados_a_treinar)):
    treinamento_x.append(dados_a_treinar[i -60: i, 0])
    treinamento_y.append(dados_a_treinar[i, 0])

    if i <= 61:
        print(treinamento_x)
        print(treinamento_y)

In [ ]:
#Adequando o array para 3 dimensões para seguir o modelo de treinamento
treinamento_x, treinamento_y = np.array(treinamento_x), np.array(treinamento_y)

treinamento_x = treinamento_x.reshape(treinamento_x.shape[0], treinamento_x.shape[1], 1)

print(treinamento_x)

In [ ]:
modelo = Sequential()

#O modelo será construído com 50 neurônios

modelo.add(LSTM(50, return_sequences=True, input_shape = (treinamento_x.shape[1], 1)))
modelo.add(LSTM(50, return_sequences=False))
modelo.add(Dense(25))
modelo.add(Dense(1))

treinamento_x.shape[1]

In [ ]:
#O modelo irá utilizar o algorítimo Adam como otimizador e terá sua função de erro medida em erro quadrático
modelo.compile(optimizer="adam", loss="mean_squared_error")

In [ ]:
#O modelo será otimizado a cada 1 iteração e neste momento só itera 1 vez sobre todo os dados
modelo.fit(treinamento_x, treinamento_y, batch_size=1, epochs=1)

In [46]:
dados_teste = dados_0_1[dados_a_treinar - 60:, :]

teste_x = []

teste_y = cotacao[dados_sendo_treinados: , :]

for i in range(60, len(dados_teste)):
    teste_x.append(dados_teste[i - 60, 0])

TypeError: only integer scalar arrays can be converted to a scalar index